<a href="https://colab.research.google.com/github/Rmostert/LLM-use-cases/blob/Audio_to_text/audio_to_text/audio_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Audio to text, translation and summarisation

In this project I'm interested in a few topics: transcribing an audio file and translating and summarizing it - all using Foundational models availible on Hugging Face.
I made us of a audio file in German talking about Skateboarding. This file (and my others) are availible under the following [link](https://germanlistening.com/download).

The following models were used:
1. [openai/whisper-large-v3](https://huggingface.co/openai/whisper-large-v3) for transcribing and translating
2. [Deepl](https://www.deepl.com/en/products/api) for translating
3. [facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn) for text summarization




In [27]:
!pip install -U transformers deepl evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


## Local Inference on GPU
For the transcription part we'll do the inference locally on a GPU

In [28]:
import deepl
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import evaluate
from google.colab import userdata
from google.colab import drive
from huggingface_hub import InferenceClient

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.mount('/content/drive')

HF_TOKEN = userdata.get('HF_TOKEN')
DEEPL_TOKEN = userdata.get('DEEPL_TOKEN')

deepl_client = deepl.DeepLClient(DEEPL_TOKEN)

client = InferenceClient(
    provider="auto",
    api_key=HF_TOKEN,
)

In [2]:
# Load model directly

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"


model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [4]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    dtype=torch_dtype,
    device=device,
    language= "german",
    return_timestamps=True
)

Device set to use cuda:0


In [5]:
sample = "/content/drive/MyDrive/audiofiles/germanlistening.com-1107-skateboarden-lernen.mp3"
result = pipe(sample)

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [34]:
result['text']

' Skateboarden lernen Skateboardfahren ist ein beliebter und verbreiteter Sport in Deutschland. Überall gibt es Skateparks oder Skatehallen. Ein Skateboard fahren zu können, setzt aber viel Training voraus. Zuerst muss man sich ein Skateboard kaufen oder ausleihen. Am besten zieht man auch Schutzausrüstung wie zum Beispiel einen Helm und Schütze an. Man braucht ausreichend Platz. Dort stellt man sich auf das Skateboard und gewöhnt sich an das Gefühl, auf dem Brett zu stehen. Mit einem Fuß schiebt man das Brett an. Man sollte langsam anfangen und ein Gefühl dafür bekommen, wie man ein Skateboard lenkt und bremst. Wenn es alleine nicht funktioniert, kann man jemanden um Hilfe bitten. Die zweite Person kann einen dann festhalten oder unterstützen. Nach einer Weile kann man sicher fahren. Dann kann man ein paar Dinge ausprobieren, zum Beispiel auf Rampen zu fahren oder springen zu lernen. Einen Sprung mit dem Skateboard zu schaffen ist schwierig, aber ein wichtiger Teil des Sports. Kann ma

In [16]:
with open("/content/drive/MyDrive/audiofiles/germanlistening.com-1107-skateboarden-lernen.txt", "r") as f:
    actual_text = f.read().replace('\n',' ')

In [17]:
actual_text

'Skateboarden lernen Skateboardfahren ist ein beliebter und verbreiteter Sport in Deutschland. Überall gibt es Skateparks oder Skatehallen. Ein Skateboard fahren zu können setzt aber viel Training voraus. Zuerst muss man sich ein Skateboard kaufen oder ausleihen. Am besten zieht man auch Schutzausrüstung wie zum Beispiel einen Helm und Schützer an. Man braucht ausreichend Platz. Dort stellt man sich auf das Skateboard und gewöhnt sich an das Gefühl auf dem Brett zu stehen. Mit einem Fuß schiebt man das Brett an. Man sollte langsam anfangen und ein Gefühl dafür bekommen, wie man ein Skateboard lenkt und bremst. Wenn es alleine nicht funktioniert, kann man jemanden um Hilfe bitten. Die zweite Person kann einen dann festhalten oder unterstützen. Nach einer Weile kann man sicher fahren. Dann kann man ein paar Dinge ausprobieren, zum Beispiel auf Rampen zu fahren oder springen zu lernen. Einen Sprung mit dem Skateboard zu schaffen ist schwierig, aber ein wichtiger Teil des Sports. Kann man 

A perfect transcription! It of couse helps that the speaker speaks slowly and clearly and that there are no background noise. We can meke use of the BLEU score to evalaute the transcription

In [ ]:
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(
    predictions=[result['text'].strip()],
    references=[actual_text]
)

In [36]:
bleu_score

{'bleu': 0.9599358886936473,
 'precisions': [0.9842105263157894,
  0.9682539682539683,
  0.9521276595744681,
  0.9358288770053476],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0106382978723405,
 'translation_length': 190,
 'reference_length': 188}

An almost perfect score.The model can also be used to automatically translated the audio into English. This will be tried next

In [19]:
translated_text = pipe(sample, generate_kwargs={"task": "translate"})

In [20]:
translated_text['text']

' learn skateboarding skateboarding is a popular and widespread sport in germany everywhere there are skate parks or skate halls to be able to ride a skateboard but requires a lot of training first you have to buy a skateboard or borrow it is best to wear protective gear such as a helmet and a shooter you need enough Platz. Dort stellt man sich auf das Skateboard und gewöhnt sich an das Gefühl, auf dem Brett zu stehen. Mit einem Fuß schiebt man das Brett an. Man sollte langsam anfangen und ein Gefühl dafür bekommen, wie man ein Skateboard lenkt und bremst. Wenn es alleine nicht funktioniert, kann man jemanden um Hilfe bitten. Die zweite Person kann einen dann festhalten oder unterstützen. Nach After a while you can ride safely. Then you can try a few things, for example to ride on ramps or learn to jump. It is difficult to make a jump with the skateboard, but an important part of the sport. If you can jump well and high, you can learn a lot of new tricks.'

This is a bit strange. The model starts okay (despite the lack of punctuation) and then stops translating. Only at the end it translates the words again. Here I'll rather use Deepl - a brilliant translator that I use regularly to translate from German to English and vice versa.

In [22]:
translated_text = deepl_client.translate_text(result['text'], target_lang="EN-GB")

In [23]:
translated_text.text

" Learn to skateboard Skateboarding is a popular and widespread sport in Germany. There are skate parks and skate halls everywhere. However, being able to ride a skateboard requires a lot of training. First of all, you need to buy or hire a skateboard. It is also best to put on protective equipment such as a helmet and protectors. You need enough space. Stand on the skateboard and get used to the feeling of standing on the board. Push the board with one foot. You should start slowly and get a feel for how to steer and brake a skateboard. If you can't do it on your own, you can ask someone for help. The second person can then hold on to you or support you. After a while, you will be able to ride safely. Then you can try out a few things, such as riding on ramps or learning to jump. Jumping with a skateboard is difficult, but an important part of the sport. If you can jump well and high, you can learn lots of new tricks."

This is a very good translation, but to get a more objective measure , you can again make us of the BLEU score to evaulate a model's performance in translation tasks. Lastly we can summarize the topic by making use of remote inference of a Hugging face model


In [24]:
result_summarization = client.summarization(
    translated_text.text,
    model="facebook/bart-large-cnn",
)

In [37]:
result_summarization.summary_text

'Skateboarding is a popular and widespread sport in Germany. There are skate parks and skate halls everywhere. Being able to ride a skateboard requires a lot of training. It is also best to put on protective equipment such as a helmet and protectors. If you can jump well and high, you can learn lots of new tricks.'

Even though it wasn't a big text, the summary is good enough